## Svenska Akademin  - Wikidata
version 2.2

WD egenskap [Property:P5325](https://www.wikidata.org/wiki/Property:P5325) 
* this [notebook](https://github.com/salgo60/open-data-examples/blob/master/Svenska%20Akademin.ipynb)  
* Wikidata [WikiProject_Sweden/Swedish_Academy](https://www.wikidata.org/wiki/Wikidata:WikiProject_Sweden/Swedish_Academy)
  * SPARQL -> [objects maintained by the project](https://w.wiki/XTB)
----

### What has been done
1. [query Wikidata](https://w.wiki/XBp) for people in the Swedish Academy
1. using [Property:P5325](https://www.wikidata.org/wiki/Property:P5325)  to access the page and webscrape some data
1. create ds with numbering and replace/replaced by
1. use Open Refine for connect it to Wikidata
1. batch upload using [Quickstatement](https://quickstatements.toolforge.org/#/)

#### Other sources we sync
* [Kulturpersoner Uppsalakyrkogård](https://github.com/salgo60/open-data-examples/blob/master/Check%20WD%20kulturpersoner%20uppsalakyrkogardar.ipynb)
* [Litteraturbanken](https://github.com/salgo60/open-data-examples/blob/master/Litteraturbanken%20Author.ipynb)
* [Nobelprize.org](https://github.com/salgo60/open-data-examples/blob/master/Nobel%20API.ipynb)
* [SBL](https://github.com/salgo60/open-data-examples/blob/master/SBL.ipynb)
* [SKBL](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%203.ipynb)  
* [Svenska Akademin](https://github.com/salgo60/open-data-examples/blob/master/Svenska%20Akademin.ipynb)  


In [1]:
from datetime import datetime
now = datetime.now()
print("Last run: ", now)

Last run:  2020-09-29 00:42:50.809588


In [2]:
import urllib3, json
import pandas as pd   
from bs4 import BeautifulSoup
import sys
import pprint
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm.notebook import trange  
from wikidataintegrator import wdi_core, wdi_login

endpoint_url = "https://query.wikidata.org/sparql"

SparqlQuery = """SELECT ?item ?svaid WHERE {
?item wdt:P5325 ?svaid
}"""

# query 2 https://w.wiki/XBp
SparqlQuery_version2 = """SELECT ?item ?svaid ?chairLabel ?chairOrder ?start ?end ?ordinalnr ?replacedby ?replacedbyLabel ?replaces ?replacesLabel WHERE {
  ?item wdt:P5325 ?svaid.
  OPTIONAL {
    ?item p:P39 ?Pchair.
    ?Pchair ps:P39 ?chair.
    ?chair wdt:P361 wd:Q207360.
    OPTIONAL { ?chair wdt:P1545 ?chairOrder}
    OPTIONAL { ?Pchair pq:P580 ?start. }
    OPTIONAL { ?Pchair pq:P582 ?end. }
    OPTIONAL { ?Pchair pq:P1365 ?replaces. }
    OPTIONAL { ?Pchair pq:P1366 ?replacedby. }
    OPTIONAL { ?Pchair pq:P1545 ?ordinalnr. }
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
} order by xsd:integer(?order) xsd:integer(?ordinalnr)"""

http = urllib3.PoolManager()

# Query https://w.wiki/Vo5
def get_results(endpoint_url, query):
    user_agent = "user  salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def getChair (sakid):
    url = "http://www.svenskaakademien.se/svenska-akademien/ledamotsregister/" + sakid
    r = http.request('GET', url) 
    soup = BeautifulSoup(r.data, "html.parser")
    try:
        stol = soup.findAll("div", {"class": "field-name-field-stol"})[0].get_text()
        stol = stol.replace(u'Stol:\xa0', u'')
    except:
        stol =""
    try: 
        intrade = soup.findAll("div", {"class": "field-name-field-tog-intrade"})[0].get_text()
        intrade = intrade.replace(u'Inträde:\xa0',u'')
    except:
        intrade =""
    try: 
        avled = soup.findAll("div", {"class": "field-name-field-avled"})[0].get_text()
        avled = avled.replace(u'Utträde:\xa0',u'')
    except:
        avled =""
    return stol,intrade,avled

SparQlResults = get_results(endpoint_url, SparqlQuery_version2)
length = len (SparQlResults["results"]["bindings"])
df = pd.DataFrame(columns=['WD', 'SvenskaAkademin', 'wdReplace', 'wdReplaceBy', 'wdStart', 'wdEnd',
                           'wdOrdinal','chairOrder','SvChair', 'SvStart', 'SvEnd'])
for r in trange(0,length):
    resultSparql = SparQlResults["results"]["bindings"][r]
    wd = resultSparql["item"]["value"].replace("http://www.wikidata.org/entity/","") 
    try:
        wdReplace = resultSparql["item"]["value"].replace("http://www.wikidata.org/entity/","") 
    except:
        wdReplace = ""
    try:
        wdReplaceBy = resultSparql["item"]["value"].replace("http://www.wikidata.org/entity/","") 
    except:
        wdReplaceBy = ""
    try: 
        wdStart = resultSparql["start"]["value"] 
    except:
        wdStart = ""    
    try:
        wdEnd = resultSparql["end"]["value"] 
    except:
        wdEnd = ""
    try:
        wdOrdinal = resultSparql["ordinalnr"]["value"] 
    except:
        wdOrdinal = ""
    try:
        chairOrder = resultSparql["chairOrder"]["value"] 
    except:
        chairOrder = ""
    try:
        sakid = resultSparql["svaid"]["value"]
    except:
        sakid = ""
    chair, start, end = getChair(sakid) 
    df = df.append({'WD': wd, 'SvChair': chair, 'SvStart': start, 'SvEnd': end, 'SvenskaAkademin': sakid, 
                   'wdReplace' : wdReplace, 'wdReplaceBy' : wdReplaceBy, 'wdStart' : wdStart, 'wdEnd' : wdEnd,
                   'wdOrdinal' : wdOrdinal, 'chairOrder': chairOrder}, ignore_index=True)
  

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   WD               221 non-null    object
 1   SvenskaAkademin  221 non-null    object
 2   wdReplace        221 non-null    object
 3   wdReplaceBy      221 non-null    object
 4   wdStart          221 non-null    object
 5   wdEnd            221 non-null    object
 6   wdOrdinal        221 non-null    object
 7   chairOrder       221 non-null    object
 8   SvChair          221 non-null    object
 9   SvStart          221 non-null    object
 10  SvEnd            221 non-null    object
dtypes: object(11)
memory usage: 19.1+ KB


In [4]:
df.head(200)

,WD,SvenskaAkademin,wdReplace,wdReplaceBy,wdStart,wdEnd,wdOrdinal,chairOrder,SvChair,SvStart,SvEnd
0,Q4252338,lagerbjelke-gustaf,Q4252338,Q4252338,1819-09-25T00:00:00Z,1837-05-24T00:00:00Z,3,10,10,1819-09-25,1837-05-24
1,Q4349660,beskow-bernhard-von,Q4349660,Q4349660,1828-11-01T00:00:00Z,1868-10-17T00:00:00Z,4,12,12,1828-11-01,1868-10-17
2,Q4349660,beskow-bernhard-von,Q4349660,Q4349660,1834-01-01T00:00:00Z,1868-01-01T00:00:00Z,3,,12,1828-11-01,1868-10-17
3,Q4355881,larsson-hans,Q4355881,Q4355881,1925-12-20T00:00:00Z,1944-02-16T00:00:00Z,7,15,15,1925-12-20,1944-02-16
4,Q4358870,gyldenstolpe-nils-philip,Q4358870,Q4358870,1789-06-18T00:00:00Z,1810-02-20T00:00:00Z,2,1,1,1789-12-11,1810-02-20
...,...,...,...,...,...,...,...,...,...,...,...
195,Q439832,lundkvist-artur,Q439832,Q439832,1968-12-20T00:00:00Z,1991-12-11T00:00:00Z,14,18,18,1968-12-20,1991-12-11
196,Q442031,vallquist-gunnel,Q442031,Q442031,1982-12-20T00:00:00Z,2016-01-11T00:00:00Z,7,13,13,1982-12-20,2016-01-11
197,Q444538,atterbom-per,Q444538,Q444538,1840-05-29T00:00:00Z,1855-07-21T00:00:00Z,4,18,18,1840-05-29,1855-07-21
198,Q465251,josephson-ragnar,Q465251,Q465251,1960-12-20T00:00:00Z,1966-03-27T00:00:00Z,7,14,14,1960-12-20,1966-03-27


In [5]:
df['wdStart'] = pd.to_datetime(df['wdStart'])
df['wdEnd'] = pd.to_datetime(df['wdEnd'])
df['wdStart'] = df['wdStart'].dt.date  
df['wdEndDate'] = df['wdEnd'].dt.date 
#df['wdStartDate'] = df['wdStart'].dt.date 
df['SvStart'] = pd.to_datetime(df['SvStart']) 
df['SvEnd'] = pd.to_datetime(df['SvEnd']) 
df['SvChair'] = df['SvChair'].astype(int)
#df['chairOrder'] = df['chairOrder'].astype(int)

In [6]:
df.head(100)


,WD,SvenskaAkademin,wdReplace,wdReplaceBy,wdStart,wdEnd,wdOrdinal,chairOrder,SvChair,SvStart,SvEnd,wdEndDate
0,Q4252338,lagerbjelke-gustaf,Q4252338,Q4252338,1819-09-25,1837-05-24 00:00:00+00:00,3,10,10,1819-09-25,1837-05-24,1837-05-24
1,Q4349660,beskow-bernhard-von,Q4349660,Q4349660,1828-11-01,1868-10-17 00:00:00+00:00,4,12,12,1828-11-01,1868-10-17,1868-10-17
2,Q4349660,beskow-bernhard-von,Q4349660,Q4349660,1834-01-01,1868-01-01 00:00:00+00:00,3,,12,1828-11-01,1868-10-17,1868-01-01
3,Q4355881,larsson-hans,Q4355881,Q4355881,1925-12-20,1944-02-16 00:00:00+00:00,7,15,15,1925-12-20,1944-02-16,1944-02-16
4,Q4358870,gyldenstolpe-nils-philip,Q4358870,Q4358870,1789-06-18,1810-02-20 00:00:00+00:00,2,1,1,1789-12-11,1810-02-20,1810-02-20
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Q3148840,sjoberg-nils-lorents,Q3148840,Q3148840,1787-05-13,1822-03-13 00:00:00+00:00,1,18,18,1787-05-13,1822-03-13,1822-03-13
96,Q3338577,adlerbeth-gudmund-goran,Q3338577,Q3338577,1786-04-05,1818-10-07 00:00:00+00:00,1,9,9,1786-04-05,1818-10-07,1818-10-07
97,Q3352137,lonnroth-erik,Q3352137,Q3352137,1962-12-20,2002-03-10 00:00:00+00:00,10,10,10,1962-12-20,2002-03-10,2002-03-10
98,Q3352340,skjoldebrand-anders-fredrik,Q3352340,Q3352340,1822-10-21,1834-08-23 00:00:00+00:00,2,18,18,1822-10-21,1834-08-23,1834-08-23


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   WD               221 non-null    object             
 1   SvenskaAkademin  221 non-null    object             
 2   wdReplace        221 non-null    object             
 3   wdReplaceBy      221 non-null    object             
 4   wdStart          221 non-null    object             
 5   wdEnd            203 non-null    datetime64[ns, UTC]
 6   wdOrdinal        221 non-null    object             
 7   chairOrder       221 non-null    object             
 8   SvChair          221 non-null    int64              
 9   SvStart          220 non-null    datetime64[ns]     
 10  SvEnd            198 non-null    datetime64[ns]     
 11  wdEndDate        203 non-null    object             
dtypes: datetime64[ns, UTC](1), datetime64[ns](2), int64(1), object(8)
memory usage

In [8]:
pd.set_option("display.max.rows", None) 

In [9]:
akstolar = df.sort_values(["SvChair","SvStart"])
akstolar

,WD,SvenskaAkademin,wdReplace,wdReplaceBy,wdStart,wdEnd,wdOrdinal,chairOrder,SvChair,SvStart,SvEnd,wdEndDate
200,Q491083,hopken-anders-johan-von,Q491083,Q491083,1786-04-05,1789-05-09 00:00:00+00:00,1,1,1,1786-04-05,1789-05-09,1789-05-09
4,Q4358870,gyldenstolpe-nils-philip,Q4358870,Q4358870,1789-06-18,1810-02-20 00:00:00+00:00,2,1,1,1789-12-11,1810-02-20,1810-02-20
132,Q1355965,wallin-johan-olof,Q1355965,Q1355965,1811-05-29,1839-06-30 00:00:00+00:00,3,1,1,1811-05-29,1839-06-30,1839-06-30
146,Q1959091,fryxell-anders,Q1959091,Q1959091,1841-05-23,1881-03-21 00:00:00+00:00,4,1,1,1841-05-23,1881-03-21,1881-03-21
76,Q2291097,forssell-hans,Q2291097,Q2291097,1881-12-20,1901-07-31 00:00:00+00:00,5,1,1,1881-12-20,1901-07-31,1901-07-31
58,Q2070822,bildt-carl,Q2070822,Q2070822,1902-06-20,1931-01-26 00:00:00+00:00,6,1,1,1902-06-20,1931-01-26,1931-01-26
74,Q2193692,wedberg-birger,Q2193692,Q2193692,1931-12-20,1945-10-02 00:00:00+00:00,7,1,1,1931-12-20,1945-10-02,1945-10-02
75,Q2286404,ekeberg-birger,Q2286404,Q2286404,1945-12-20,1968-11-30 00:00:00+00:00,8,1,1,1945-12-20,1968-11-30,1968-11-30
109,Q736172,petren-sture,Q736172,Q736172,1969-12-20,1976-12-13 00:00:00+00:00,9,1,1,1969-12-20,1976-12-13,1976-12-13
134,Q1358507,rudholm-sten,Q1358507,Q1358507,1977-12-20,2008-11-29 00:00:00+00:00,10,1,1,1977-12-20,2008-11-29,2008-11-29


In [10]:
akstolar = akstolar.reset_index()


In [11]:
stolar = {1: "Q96599799", 2: "Q96600293",3: "Q96600298", 4: "Q96600304", 5:"Q96600314",
         6: "Q96600324", 7: "Q96600329", 8: "Q96600338", 9: "Q96600350", 10:"Q96600362",
         11: "Q96600367", 12: "Q96600377", 13: "Q96600389", 14: "Q96600400", 15: "Q96600412",
         16: "Q96600424", 17: "Q96600443", 18:"Q96600458"}

In [12]:
akstolar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   index            221 non-null    int64              
 1   WD               221 non-null    object             
 2   SvenskaAkademin  221 non-null    object             
 3   wdReplace        221 non-null    object             
 4   wdReplaceBy      221 non-null    object             
 5   wdStart          221 non-null    object             
 6   wdEnd            203 non-null    datetime64[ns, UTC]
 7   wdOrdinal        221 non-null    object             
 8   chairOrder       221 non-null    object             
 9   SvChair          221 non-null    int64              
 10  SvStart          220 non-null    datetime64[ns]     
 11  SvEnd            198 non-null    datetime64[ns]     
 12  wdEndDate        203 non-null    object             
dtypes: datetime64[ns, UT

In [13]:
# add columns for WD Object Chair and replace and replaced by
orderNumber = 0
for i, row in enumerate(akstolar.itertuples()):
    beforeId = i-1 
    replaceWD = ""
    replacedByWD = ""
    if akstolar.iloc[beforeId]["SvChair"] == akstolar.iloc[i]["SvChair"]:
        #print("\t",beforeId,"\t",akstolar.iloc[beforeId]["SvenskaAkademin"], akstolar.iloc[beforeId]["SvChair"])
        replaceWD = akstolar.iloc[beforeId]["WD"]
        orderNumber = orderNumber + 1 
        akstolar.at[i,"replaceWD"] = replaceWD
    else:
        orderNumber = 1
    akstolar.at[i,"orderNumber"] = orderNumber
    afterId  = i+1 
    if afterId < len(akstolar):
        if akstolar.iloc[afterId]["SvChair"] == akstolar.iloc[i]["SvChair"]:
            #print("\t",afterId,"\t",akstolar.iloc[afterId]["SvenskaAkademin"], akstolar.iloc[afterId]["SvChair"])
            replacedByWD = akstolar.iloc[afterId]["WD"]
            akstolar.at[i,"replacedByWD"] = replacedByWD
    print(i,row.WD,row.SvChair,orderNumber,row.SvenskaAkademin,row.SvStart,row.SvEnd,replaceWD,replacedByWD)
    akstolar.at[i,"newWDChair"] = stolar[row.SvChair] 
    akstolar.at[i,"orderNumber"] = orderNumber

0 Q491083 1 1 hopken-anders-johan-von 1786-04-05 00:00:00 1789-05-09 00:00:00  Q4358870
1 Q4358870 1 2 gyldenstolpe-nils-philip 1789-12-11 00:00:00 1810-02-20 00:00:00 Q491083 Q1355965
2 Q1355965 1 3 wallin-johan-olof 1811-05-29 00:00:00 1839-06-30 00:00:00 Q4358870 Q1959091
3 Q1959091 1 4 fryxell-anders 1841-05-23 00:00:00 1881-03-21 00:00:00 Q1355965 Q2291097
4 Q2291097 1 5 forssell-hans 1881-12-20 00:00:00 1901-07-31 00:00:00 Q1959091 Q2070822
5 Q2070822 1 6 bildt-carl 1902-06-20 00:00:00 1931-01-26 00:00:00 Q2291097 Q2193692
6 Q2193692 1 7 wedberg-birger 1931-12-20 00:00:00 1945-10-02 00:00:00 Q2070822 Q2286404
7 Q2286404 1 8 ekeberg-birger 1945-12-20 00:00:00 1968-11-30 00:00:00 Q2193692 Q736172
8 Q736172 1 9 petren-sture 1969-12-20 00:00:00 1976-12-13 00:00:00 Q2286404 Q1358507
9 Q1358507 1 10 rudholm-sten 1977-12-20 00:00:00 2008-11-29 00:00:00 Q736172 Q264036
10 Q264036 1 11 lotass-lotta 2009-12-20 00:00:00 2018-05-07 00:00:00 Q1358507 Q55338841
11 Q55338841 1 12 runesson-eric-

125 Q134641 11 8 karlfeldt-erik-axel 1904-12-20 00:00:00 1931-04-08 00:00:00 Q134641 Q5734962
126 Q5734962 11 9 fogelqvist-torsten 1931-12-20 00:00:00 1941-01-23 00:00:00 Q134641 Q1777178
127 Q1777178 11 10 ahnlund-nils 1941-12-20 00:00:00 1957-01-11 00:00:00 Q5734962 Q131326
128 Q131326 11 11 johnson-eyvind 1957-12-20 00:00:00 1976-08-25 00:00:00 Q1777178 Q1893659
129 Q1893659 11 12 linde-ulf 1977-12-20 00:00:00 2013-10-12 00:00:00 Q131326 Q540344
130 Q540344 11 13 ostergren-klas 2014-12-20 00:00:00 2018-05-03 00:00:00 Q1893659 Q16633253
131 Q16633253 11 14 malm-mats 2018-12-20 00:00:00 NaT Q540344 Q16633253
132 Q16633253 11 15 malm-mats 2018-12-20 00:00:00 NaT Q16633253 
133 Q6169392 12 1 schroderheim-fore-adlandet-schroder-elis 1786-04-05 00:00:00 1795-08-30 00:00:00  Q5579332
134 Q5579332 12 2 blom-isac 1797-11-25 00:00:00 1826-05-06 00:00:00 Q6169392 Q6244863
135 Q6244863 12 3 wirsen-gustaf-fredrik 1827-03-26 00:00:00 1827-12-09 00:00:00 Q5579332 Q4349660
136 Q4349660 12 4 beskow-

In [14]:
akstolar['orderNumber'] = pd.to_numeric(akstolar['orderNumber'],downcast='integer') 
akstolar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   index            221 non-null    int64              
 1   WD               221 non-null    object             
 2   SvenskaAkademin  221 non-null    object             
 3   wdReplace        221 non-null    object             
 4   wdReplaceBy      221 non-null    object             
 5   wdStart          221 non-null    object             
 6   wdEnd            203 non-null    datetime64[ns, UTC]
 7   wdOrdinal        221 non-null    object             
 8   chairOrder       221 non-null    object             
 9   SvChair          221 non-null    int64              
 10  SvStart          220 non-null    datetime64[ns]     
 11  SvEnd            198 non-null    datetime64[ns]     
 12  wdEndDate        203 non-null    object             
 13  orderNumber      221

In [15]:
akstolar

,index,WD,SvenskaAkademin,wdReplace,wdReplaceBy,wdStart,wdEnd,wdOrdinal,chairOrder,SvChair,SvStart,SvEnd,wdEndDate,orderNumber,replacedByWD,newWDChair,replaceWD
0,200,Q491083,hopken-anders-johan-von,Q491083,Q491083,1786-04-05,1789-05-09 00:00:00+00:00,1,1,1,1786-04-05,1789-05-09,1789-05-09,1,Q4358870,Q96599799,NaN
1,4,Q4358870,gyldenstolpe-nils-philip,Q4358870,Q4358870,1789-06-18,1810-02-20 00:00:00+00:00,2,1,1,1789-12-11,1810-02-20,1810-02-20,2,Q1355965,Q96599799,Q491083
2,132,Q1355965,wallin-johan-olof,Q1355965,Q1355965,1811-05-29,1839-06-30 00:00:00+00:00,3,1,1,1811-05-29,1839-06-30,1839-06-30,3,Q1959091,Q96599799,Q4358870
3,146,Q1959091,fryxell-anders,Q1959091,Q1959091,1841-05-23,1881-03-21 00:00:00+00:00,4,1,1,1841-05-23,1881-03-21,1881-03-21,4,Q2291097,Q96599799,Q1355965
4,76,Q2291097,forssell-hans,Q2291097,Q2291097,1881-12-20,1901-07-31 00:00:00+00:00,5,1,1,1881-12-20,1901-07-31,1901-07-31,5,Q2070822,Q96599799,Q1959091
5,58,Q2070822,bildt-carl,Q2070822,Q2070822,1902-06-20,1931-01-26 00:00:00+00:00,6,1,1,1902-06-20,1931-01-26,1931-01-26,6,Q2193692,Q96599799,Q2291097
6,74,Q2193692,wedberg-birger,Q2193692,Q2193692,1931-12-20,1945-10-02 00:00:00+00:00,7,1,1,1931-12-20,1945-10-02,1945-10-02,7,Q2286404,Q96599799,Q2070822
7,75,Q2286404,ekeberg-birger,Q2286404,Q2286404,1945-12-20,1968-11-30 00:00:00+00:00,8,1,1,1945-12-20,1968-11-30,1968-11-30,8,Q736172,Q96599799,Q2193692
8,109,Q736172,petren-sture,Q736172,Q736172,1969-12-20,1976-12-13 00:00:00+00:00,9,1,1,1969-12-20,1976-12-13,1976-12-13,9,Q1358507,Q96599799,Q2286404
9,134,Q1358507,rudholm-sten,Q1358507,Q1358507,1977-12-20,2008-11-29 00:00:00+00:00,10,1,1,1977-12-20,2008-11-29,2008-11-29,10,Q264036,Q96599799,Q736172


In [16]:
akstolar.loc[:,("WD","SvenskaAkademin","newWDChair","SvStart","SvEnd","orderNumber","replaceWD","replacedByWD")]

,WD,SvenskaAkademin,newWDChair,SvStart,SvEnd,orderNumber,replaceWD,replacedByWD
0,Q491083,hopken-anders-johan-von,Q96599799,1786-04-05,1789-05-09,1,NaN,Q4358870
1,Q4358870,gyldenstolpe-nils-philip,Q96599799,1789-12-11,1810-02-20,2,Q491083,Q1355965
2,Q1355965,wallin-johan-olof,Q96599799,1811-05-29,1839-06-30,3,Q4358870,Q1959091
3,Q1959091,fryxell-anders,Q96599799,1841-05-23,1881-03-21,4,Q1355965,Q2291097
4,Q2291097,forssell-hans,Q96599799,1881-12-20,1901-07-31,5,Q1959091,Q2070822
5,Q2070822,bildt-carl,Q96599799,1902-06-20,1931-01-26,6,Q2291097,Q2193692
6,Q2193692,wedberg-birger,Q96599799,1931-12-20,1945-10-02,7,Q2070822,Q2286404
7,Q2286404,ekeberg-birger,Q96599799,1945-12-20,1968-11-30,8,Q2193692,Q736172
8,Q736172,petren-sture,Q96599799,1969-12-20,1976-12-13,9,Q2286404,Q1358507
9,Q1358507,rudholm-sten,Q96599799,1977-12-20,2008-11-29,10,Q736172,Q264036
